In [1]:
import uiautomator2 as ui2
import os
import pandas as pd
import time
import unittest
import json
import re
import csv

In [19]:
def level2redcode(code_list, frequency=38000, state=305):
    listValue = []
    listValue1 = []
    if isinstance(code_list, str):
        code_list = eval(code_list)
#     list1 = code_list[2:]  # 去掉频率和长度
    list1 = code_list  # 去掉频率和长度
    if not list1:
        return 38000, []
    for li in range(len(list1)):
        n = round(int(list1[li]) / state)
        if (n > 255):
            listValue1.append(0)
            listValue1.append(n >> 8 & 0xff)
            listValue1.append(n & 0xff)
        else:
            listValue1.append(n)
    q1 = round(int(frequency / 1000), 0)  # 要取小数将第一个int改为float，最后0改要求小数位
    if (q1 > 255):
        listValue.append(q1 >> 8 & 0xff)
        listValue.append(q1 & 0xff)
    else:
        listValue.append(q1)
        listValue.append(0)
    if (len(listValue1) > 255):
        listValue.append(len(listValue1) & 0xff)
        listValue.append(len(listValue1) >> 8 & 0xff)
    else:
        listValue.append(len(listValue1))
        listValue.append(0)
    listValue = listValue + listValue1
    return listValue

In [20]:
def get_redcode():
    fr = open("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt", "r")
    levelcode = fr.read().split("\n")[-2]
    fr.close()
    redcode = level2redcode([int(i) for i in levelcode.split(" ")])
    return redcode

# 抓取第一套

In [25]:
success_brand = []
state2_brand = []

In [42]:
serial = 'd06ec202'
d = ui2.connect_usb(serial)
# d(scrollable=True).scroll(steps=10)

# 关闭APP
d.app_stop("co.leanremote.universalremotecontrol.remotecontrol")
# 启动App
d.app_start("co.leanremote.universalremotecontrol.remotecontrol")

time.sleep(2)

fr = open("空调品牌.json", 'r')
brand_list = json.loads(fr.read())
fr.close()

fr = open("空调品牌—红码状态.json", 'r')
brand_redcode_status = json.loads(fr.read())
fr.close()

d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
d.xpath(
    '//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
time.sleep(1)

for i in range(len(brand_list)):
    brand = brand_list[i]
    # 重新抓取
    if brand not in ['Acson','Kt02-d001','Uni-air','Pinshangxinhuabao','Pereg','Acsom','Goldair','First','Mingxingboyin','Yuesheng','Mitsuka','Zhengzhoujinda','Alpin']:
        continue
#     if brand_redcode_status[brand] == 0 or brand not in need_add_brand:
#         continue
#     if brand in success_brand or brand in state2_brand:
#         continue
    print(brand)
    try:
        for j in range(i):
            d.swipe(0.045, 0.38221, 0.045, 0.311)
        d.xpath('//*[@text="{}"]'.format(brand)).click()
        time.sleep(3)
        while True:
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
                break
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
                break
            time.sleep(2)
        # 确认第一套面板
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
        elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()
        else:
            print(False)
            continue
        time.sleep(1)
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/working").click()
        time.sleep(2)
        while True:
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").exists:
                break
            time.sleep(2)

        # 抓取面板开始
        # 面板类型1,有制热,初始状态是 模式制冷, 温度16, 风速1
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").exists:
            print("    面板类型为1, 抓取该面板开始~")
            rel_data = []
            # 加入关机键
            if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").click()
            time.sleep(1)
            redcode=str(get_redcode())
            rel_data.append(['close','cool','speed1','auto',27,redcode])
            # 其他开机状态
            temp_min = int(d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/display").get_text().replace("°C", ""))
            for mode in ['cool', 'heat']:
                for speed in ['speed1', 'speed2', 'speed3', 'auto']:

                    for temp in range(temp_min, 30):
                        if temp == 16:
                            if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                                os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                            time.sleep(1)
                            redcode=str(get_redcode())
                            rel_data.append(['open',mode,speed,'auto',temp,redcode])

                        if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                            os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Up").click()
                        time.sleep(1)
                        redcode=str(get_redcode())
                        rel_data.append(['open',mode,speed,'auto',temp+1,redcode])

                        time.sleep(1)
                    # 将温度重新退回16度
                    for i in range(16):
                        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                        time.sleep(1)
                    if speed == 'auto':
                        break
                    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").click()
                    time.sleep(1)

                if mode == 'heat':
                    break
                for i in range(4):
                    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Down").click()
                    time.sleep(1)
                d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/mode").click()
                time.sleep(1)
            df = pd.DataFrame(rel_data, columns=['state','mode','speed','direct','temperature','redcode'])
            df.to_csv("./rel_data/{}_1.csv".format(brand), index=False)
#             success_brand.append(brand)
            print("      品牌抓取成功~")
        # 面板类型2
        # 面板类型2,没有制热,初始状态是 没有显示模式和风速,并且风速不是状态码,独立的, 跳过,暂时不抓
        else:
            print("    面板类型为2, 跳过~")
#             state2_brand.append(brand)
        # 返回到设备选择界面
        d(resourceId="com.android.systemui:id/back").click()
        time.sleep(1)
        d(resourceId="com.android.systemui:id/back").click()

        # 重新进入品牌选择界面
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
        d.xpath('//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
        time.sleep(1)
        
    except Exception as e:
        print(e)
        print("    品牌抓取失败~")
        # 重启app
        # 关闭APP
        d.app_stop("co.leanremote.universalremotecontrol.remotecontrol")
        # 启动App
        d.app_start("co.leanremote.universalremotecontrol.remotecontrol")

        time.sleep(2)

        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
        d.xpath(
            '//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
        time.sleep(1)

[D 200527 09:42:44 __init__:590] kill process(ps): uiautomator
[D 200527 09:42:45 __init__:608] uiautomator-v2 is starting ... left: 40.0s
[D 200527 09:42:46 __init__:608] uiautomator-v2 is starting ... left: 38.9s
[D 200527 09:42:47 __init__:608] uiautomator-v2 is starting ... left: 37.9s
[D 200527 09:42:48 __init__:608] uiautomator-v2 is starting ... left: 36.8s
[D 200527 09:42:49 __init__:608] uiautomator-v2 is starting ... left: 35.8s
[D 200527 09:42:50 __init__:608] uiautomator-v2 is starting ... left: 34.7s
[D 200527 09:42:51 __init__:608] uiautomator-v2 is starting ... left: 33.7s
[I 200527 09:42:51 __init__:573] uiautomator back to normal


Uni-air
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Acsom
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Acson
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Alpin
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
First
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Goldair
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Kt02-d001
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Mingxingboyin
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Mitsuka
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Pereg
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Pinshangxinhuabao
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Yuesheng


KeyboardInterrupt: 

# 抓取第二套

In [74]:
success_brand = []
state2_brand = []

In [83]:
serial = '1333e1d50606'
d = ui2.connect_usb(serial)
# d(scrollable=True).scroll(steps=10)

# 关闭APP
d.app_stop("co.leanremote.universalremotecontrol.remotecontrol")
# 启动App
d.app_start("co.leanremote.universalremotecontrol.remotecontrol")

time.sleep(2)

fr = open("空调品牌.json", 'r')
brand_list = json.loads(fr.read())
fr.close()

fr = open("空调品牌—红码状态.json", 'r')
brand_redcode_status = json.loads(fr.read())
fr.close()

d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
d.xpath(
    '//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
time.sleep(1)

for i in range(len(brand_list)):
    brand = brand_list[i]
    # 重新抓取
#     if brand not in ['Acson','Kt02-d001','Uni-air','Pinshangxinhuabao','Pereg','Acsom','Goldair','First','Mingxingboyin','Yuesheng','Mitsuka','Zhengzhoujinda','Alpin']:
#         continue
    if brand_redcode_status[brand] == 0 or brand not in need_add_brand:
        continue
    if brand in success_brand or brand in state2_brand:
        continue
    print(brand)
    try:
        for j in range(i):
            d.swipe(0.045, 0.38221, 0.045, 0.311)
        d.xpath('//*[@text="{}"]'.format(brand)).click()
        time.sleep(3)
        while True:
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
                break
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
                break
            time.sleep(2)

        # 确认第二套面板
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
        elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()

        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/notworking").click()
        time.sleep(2)
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
        elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()
        else:
            print(3, False, "该品牌没有第二套")
            success_brand.append(brand)
            continue

        time.sleep(1)
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/working").click()
        time.sleep(2)
        while True:
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").exists:
                break
            time.sleep(2)

        # 抓取面板开始
        # 面板类型1,有制热,初始状态是 模式制冷, 温度16, 风速1
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").exists:
            print("    面板类型为1, 抓取该面板开始~")
            rel_data = []
            # 加入关机键
            if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").click()
            time.sleep(1)
            redcode=str(get_redcode())
            rel_data.append(['close','cool','speed1','auto',27,redcode])
            # 其他开机状态
            temp_min = int(d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/display").get_text().replace("°C", ""))
            for mode in ['cool', 'heat']:
                for speed in ['speed1', 'speed2', 'speed3', 'auto']:

                    for temp in range(temp_min, 30):
                        if temp == 16:
                            if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                                os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                            time.sleep(1)
                            redcode=str(get_redcode())
                            rel_data.append(['open',mode,speed,'auto',temp,redcode])

                        if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                            os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Up").click()
                        time.sleep(1)
                        redcode=str(get_redcode())
                        rel_data.append(['open',mode,speed,'auto',temp+1,redcode])

                        time.sleep(1)
                    # 将温度重新退回16度
                    for i in range(16):
                        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                        time.sleep(1)
                    if speed == 'auto':
                        break
                    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").click()
                    time.sleep(1)

                if mode == 'heat':
                    break
                for i in range(4):
                    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Down").click()
                    time.sleep(1)
                d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/mode").click()
                time.sleep(1)
            df = pd.DataFrame(rel_data, columns=['state','mode','speed','direct','temperature','redcode'])
            
            # 注意修改保存路径
            df.to_csv("./rel_data2/{}_2.csv".format(brand), index=False)
            success_brand.append(brand)
            print("      品牌抓取成功~")
        # 面板类型2
        # 面板类型2,没有制热,初始状态是 没有显示模式和风速,并且风速不是状态码,独立的, 跳过,暂时不抓
        else:
            print("    面板类型为2, 跳过~")
            state2_brand.append(brand)
        # 返回到设备选择界面
        d(resourceId="com.android.systemui:id/back").click()
        time.sleep(1)
        d(resourceId="com.android.systemui:id/back").click()

        # 重新进入品牌选择界面
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
        d.xpath('//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
        time.sleep(1)
        
    except Exception as e:
        print(e)
        print("    品牌抓取失败~")
        # 重启app
        # 关闭APP
        d.app_stop("co.leanremote.universalremotecontrol.remotecontrol")
        # 启动App
        d.app_start("co.leanremote.universalremotecontrol.remotecontrol")

        time.sleep(2)

        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
        d.xpath(
            '//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
        time.sleep(1)

Beaver
    面板类型为1, 抓取该面板开始~
[Errno 2] No such file or directory: 'C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt'
    品牌抓取失败~
Dunham-Bush
    面板类型为1, 抓取该面板开始~
[Errno 2] No such file or directory: 'C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt'
    品牌抓取失败~
Shuanglu
    面板类型为1, 抓取该面板开始~
[Errno 2] No such file or directory: 'C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt'
    品牌抓取失败~


# 抓取第三套

In [100]:
success_brand = []
state2_brand = []

In [202]:
serial = '1333e1d50606'
d = ui2.connect_usb(serial)
# d(scrollable=True).scroll(steps=10)

# 关闭APP
d.app_stop("co.leanremote.universalremotecontrol.remotecontrol")
# 启动App
d.app_start("co.leanremote.universalremotecontrol.remotecontrol")

time.sleep(2)

fr = open("空调品牌.json", 'r')
brand_list = json.loads(fr.read())
fr.close()

fr = open("空调品牌—红码状态.json", 'r')
brand_redcode_status = json.loads(fr.read())
fr.close()

d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
d.xpath(
    '//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
time.sleep(1)

for i in range(len(brand_list)):
    brand = brand_list[i]
    # 重新抓取
    if brand not in ['Pinshangxinhuabao', 'Uni-air', 'SASY', 'DIY', 'Whisper Aire', 'Polytron', 'Pereg', 'Yorkyork', 'Xindongbao', 'Goldair', 'Chengyuan', 'Gelin', 'Acsom', 'Polka', 'Starlight', 'Unionaire', 'Xiongdi', 'Senville', 'Shamei', 'Gaoluhua', 'Mingxingboyin', 'Starway', 'Tobo', 'Xinfei', 'Mirage', 'Sowa', 'Yuesheng', 'Electer', 'Reetech', 'Alpin', 'Yingyan', 'Zhengzhoujinda', 'Brastemp', 'Comfee', 'Tongli', 'Videocon']:   
        continue
#     if brand_redcode_status[brand] == 0 or brand not in need_add_brand:
#         continue
#     if brand in success_brand or brand in state2_brand:
#         continue
    print(brand)
    try:
        for j in range(i):
            d.swipe(0.045, 0.38221, 0.045, 0.311)
        d.xpath('//*[@text="{}"]'.format(brand)).click()
        time.sleep(3)
        while True:
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
                break
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
                break
            time.sleep(2)

        # 确认第三套面板
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
        elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()
        time.sleep(1)
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/notworking").click()
        time.sleep(5)
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
        elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()
        else:        
            print(3, False, "该品牌没有第二套, 自然没第三套")
            success_brand.append(brand)
            continue
        time.sleep(1)
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/notworking").click()
        time.sleep(5)
        
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOn1").click()
        elif d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").exists:
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/power").click()
        else:
            print(3, False, "该品牌没有第三套")
            success_brand.append(brand)
            continue

        time.sleep(1)
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/working").click()
        time.sleep(5)
        while True:
            if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").exists:
                break
            time.sleep(2)

        # 抓取面板开始
        # 面板类型1,有制热,初始状态是 模式制冷, 温度16, 风速1
        if d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").exists:
            print("    面板类型为1, 抓取该面板开始~")
            rel_data = []
            # 加入关机键
            if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/PowerOff").click()
            time.sleep(1)
            redcode=str(get_redcode())
            rel_data.append(['close','cool','speed1','auto',27,redcode])
            # 其他开机状态
            temp_min = int(d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/display").get_text().replace("°C", ""))
            for mode in ['cool', 'heat']:
                for speed in ['speed1', 'speed2', 'speed3', 'auto']:

                    for temp in range(temp_min, 30):
                        if temp == 16:
                            if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                                os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                            d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                            time.sleep(1)
                            redcode=str(get_redcode())
                            rel_data.append(['open',mode,speed,'auto',temp,redcode])

                        if os.path.exists("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt"):
                            os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
                        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Up").click()
                        time.sleep(1)
                        redcode=str(get_redcode())
                        rel_data.append(['open',mode,speed,'auto',temp+1,redcode])

                        time.sleep(1)
                    # 将温度重新退回16度
                    for i in range(16):
                        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Volume_Down").click()
                        time.sleep(1)
                    if speed == 'auto':
                        break
                    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Up").click()
                    time.sleep(1)

                if mode == 'heat':
                    break
                for i in range(4):
                    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Channel_Down").click()
                    time.sleep(1)
                d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/mode").click()
                time.sleep(1)
            df = pd.DataFrame(rel_data, columns=['state','mode','speed','direct','temperature','redcode'])
            
            # 注意修改保存路径
            df.to_csv("./rel_data3/{}_3.csv".format(brand), index=False)
            success_brand.append(brand)
            print("      品牌抓取成功~")
        # 面板类型2
        # 面板类型2,没有制热,初始状态是 没有显示模式和风速,并且风速不是状态码,独立的, 跳过,暂时不抓
        else:
            print("    面板类型为2, 跳过~")
            state2_brand.append(brand)
        # 返回到设备选择界面
        d(resourceId="com.android.systemui:id/back").click()
        time.sleep(1)
        d(resourceId="com.android.systemui:id/back").click()

        # 重新进入品牌选择界面
        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
        d.xpath('//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
        time.sleep(1)
        
    except Exception as e:
        print(e)
        print("    品牌抓取失败~")
        # 重启app
        # 关闭APP
        d.app_stop("co.leanremote.universalremotecontrol.remotecontrol")
        # 启动App
        d.app_start("co.leanremote.universalremotecontrol.remotecontrol")

        time.sleep(2)

        d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/default_text_view", text="IR Remote").click()
        d.xpath(
            '//*[@resource-id="co.leanremote.universalremotecontrol.remotecontrol:id/gridview"]/android.widget.LinearLayout[2]/android.widget.ImageView[1]').click()
        time.sleep(1)

Uni-air
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Videocon
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Acsom
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Alpin
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Brastemp
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Chengyuan
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Comfee
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
DIY
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Electer
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Gaoluhua
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Gelin
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Goldair
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Mingxingboyin
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Mirage
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Pereg
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Pinshangxinhuabao
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Polka
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Polytron
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Reetech
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
SASY
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Senville
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Shamei
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Sowa
    面板类型为1, 抓取该面板开始~
      品牌抓取成功~
Starlight
 

In [110]:
# Singer 没有关机码
# 
success_brand

['Lloyd',
 'Godrej',
 'O-General',
 'Pioneer',
 'Premium',
 'Sansui',
 'Uni-air',
 'Videocon',
 'Acson',
 'Agratto',
 'Airwave',
 'Alliance',
 'Alpin',
 'Azure',
 'BGH',
 'Boshi',
 'Bosko',
 'Brastemp',
 'Chengyuan',
 'Ciac',
 'Comfee',
 'Crafft',
 'Dayu',
 'Defy',
 'DIY',
 'Ecox',
 'Electer',
 'First',
 'Fujiaire',
 'Gaoluhua',
 'Gelin',
 'Goldair',
 'Haceb',
 'Hampton Bay',
 'Huikang',
 'Idec',
 'Jiangnan',
 'Kenmore',
 'Kolin',
 'Komeco',
 'Mingxingboyin',
 'Mirage',
 'Mitsuka',
 'Norca',
 'Nordstar',
 'Pereg',
 'Pinshangxinhuabao',
 'Polka',
 'Polytron',
 'Raybo',
 'Reetech',
 'Rolsen',
 'Sacon',
 'SASY',
 'Senville',
 'Shamei',
 'Sowa',
 'Starlight',
 'Starway',
 'Tgm',
 'Tobo',
 'Tongli',
 'Unionaire',
 'Vaillant',
 'Wangzi',
 'Whisper Aire',
 'Xindongbao',
 'Xinfei',
 'Xiongdi',
 'Yingyan',
 'Yorkyork',
 'Yuesheng',
 'Zhengzhoujinda',
 'Acsom',
 'Airfel',
 'Alaska',
 'Borler',
 'Elegance',
 'Fresh',
 'Funiki',
 'Hyunda',
 'Ivt',
 'Klimatair',
 'Kt02-d001',
 'Rayb0',
 'Yangzi',
 

In [104]:
code = [115, 112, 30, 84, 30, 27, 29, 85, 29, 85, 29, 85, 29, 84, 30, 84, 30, 85, 29, 84, 30, 27, 29, 85, 29, 85, 29, 85, 29, 84, 30, 85, 29, 84, 30, 27, 29, 27, 29, 85, 29, 27, 29, 27, 29, 27, 29, 27, 29, 27, 29, 27, 29, 27, 29, 84, 30, 27, 29, 27, 29, 27, 29, 27, 29, 27, 114, 113, 29, 85, 29, 27, 29, 85, 29, 84, 30, 85, 29, 84, 30, 85, 29, 85, 29, 85, 29, 27, 29, 84, 30, 84, 30, 84, 30, 85, 29, 85, 29, 85, 29, 27, 29, 27, 29, 84, 30, 27, 29, 27, 29, 27, 29, 27, 29, 27, 29, 27, 29, 27, 29, 85, 29, 27, 29, 27, 29, 27, 29, 27, 29, 27, 114, 112, 31, 0, 1, 196, 115, 112, 30, 27, 29, 27, 29, 84, 30, 27, 29, 85, 29, 84, 30, 85, 29, 84, 30, 27, 29, 27, 29, 85, 29, 27, 29, 84, 30, 85, 29, 85, 29, 85, 29, 27, 29, 84, 30, 84, 30, 27, 29, 85, 29, 85, 29, 27, 29, 27, 29, 27, 29, 85, 29, 84, 30, 27, 29, 85, 29, 166, 29, 27, 29, 27, 114, 112, 30, 27, 29, 27, 29, 84, 30, 27, 29, 85, 29, 85, 29, 84, 30, 85, 29, 27, 29, 27, 29, 85, 29, 27, 29, 85, 29, 85, 29, 85, 29, 85, 29, 27, 29, 84, 30, 84, 30, 27, 29, 85, 29, 85, 29, 27, 29, 27, 29, 27, 29, 85, 29, 84, 30, 27, 29, 84, 30, 84, 30, 27, 29, 27, 114, 112, 31, 0, 12, 202]
i = 0
data = []
while i < len(code):
    if code[i] == 0:
        data.append(int((code[i+1] * 256 + code[i + 2]) * 30.5))
        i += 3
    else:
        data.append(int(code[i] * 30.5))
        i += 1

In [ ]:
print(data)

In [208]:
redcode=str(get_redcode())
os.remove("C:/Users/dell/Desktop/IRReader/IRReader2015/demo20190619-1/IRDemo/bin/Debug/err_code.txt")
print(redcode)

[38, 0, 170, 0, 65, 57, 15, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 55, 15, 29, 14, 29, 14, 29, 14, 55, 15, 56, 14, 29, 14, 56, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 56, 14, 29, 14, 55, 15, 55, 15, 29, 14, 29, 14, 29, 14, 29, 14, 56, 14, 29, 14, 56, 14, 56, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 72, 64, 57, 15, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 56, 14, 29, 14, 29, 14, 29, 14, 56, 14, 55, 15, 29, 14, 56, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 56, 14, 29, 14, 55, 15, 56, 14, 29, 14, 29, 14, 29, 14, 29, 14, 56, 14, 29, 14, 56, 14, 55, 15, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 29, 14, 0, 12, 204]


25.965999999999998

'0x38'

In [203]:
# 删除保存的空调码
for i in range(39):
    d(resourceId="co.leanremote.universalremotecontrol.remotecontrol:id/Delete").click()
    d(resourceId="android:id/button1").click()

[D 200601 10:08:55 __init__:590] kill process(ps): uiautomator
[D 200601 10:08:56 __init__:608] uiautomator-v2 is starting ... left: 40.0s
[D 200601 10:08:57 __init__:608] uiautomator-v2 is starting ... left: 38.9s
[D 200601 10:08:58 __init__:608] uiautomator-v2 is starting ... left: 37.9s
[D 200601 10:09:00 __init__:608] uiautomator-v2 is starting ... left: 36.8s
[D 200601 10:09:01 __init__:608] uiautomator-v2 is starting ... left: 35.8s
[D 200601 10:09:02 __init__:608] uiautomator-v2 is starting ... left: 34.7s
[D 200601 10:09:03 __init__:608] uiautomator-v2 is starting ... left: 33.7s
[D 200601 10:09:04 __init__:608] uiautomator-v2 is starting ... left: 32.6s
[D 200601 10:09:05 __init__:608] uiautomator-v2 is starting ... left: 31.6s
[I 200601 10:09:05 __init__:573] uiautomator back to normal
